In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent))

In [ ]:
from katacv.utils.related_pkgs.jax_flax_optax_orbax import *
from katacv.ocr.parser import get_args_and_writer
from katacv.ocr.crnn_model import get_ocr_crnn_state

args = get_args_and_writer(no_writer=True, input_args="")
state = get_ocr_crnn_state(args)
weights = ocp.PyTreeCheckpointer().restore("/home/wty/Coding/models/OCR/OCR-CRNN-0020-lite")
print(weights.keys())
# weights = ocp.PyTreeCheckpointer().restore("/home/yy/Coding/models/YOLOv1/YOLOv1-0080-lite")
state = state.replace(params=weights['params'], batch_stats=weights['batch_stats'])
# state = state.replace(params=weights['params'], params_darknet=None, batch_stats=weights['batch_stats'])

In [ ]:
from katacv.ocr.ocr_predict import predict_result
import time
start_time = time.time()
print("XLA compile...")
input_shape = (1, 32, 100, 1)
predict = partial(predict_result, state=state, max_len=args.max_label_length, idx2ch=args.idx2ch)
predict(x=jnp.empty(input_shape, dtype=jnp.uint8))
print(f"Compile complete! Use time: {time.time() - start_time:.2f} s")

In [ ]:
from PIL import Image
x = jnp.array(Image.open(r"/home/wty/Coding/datasets/mjsynth/1/1/1_pontifically_58805.jpg").convert("L").resize((100,32)))[None, ..., None]
import matplotlib.pyplot as plt
plt.imshow(x[0], cmap='gray')
print(x.shape)
print(predict(x=x))

In [ ]:
import io, time
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt

upload_button = widgets.FileUpload(description="上传图片")
aux_output = widgets.Text("模型/识别用时：")

# 显示图片和调整亮度、对比度的函数
def show_image(upload):
    if upload:
        uploaded_file = upload[0]
        origin_image = Image.open(io.BytesIO(uploaded_file['content'])).convert('L')
        resize_image = origin_image.resize((100,32))
        x = jnp.array(resize_image)[None,...,None].astype('uint8')
        start_time = time.time()
        pred_result = predict(x=x)[0]
        print("Predict result:", pred_result)
        aux_output.value = f"模型用时：{time.time() - start_time:.4f} s"
        
        fig, ax = plt.subplots(figsize=(8,3))
        ax.imshow(origin_image, cmap='gray')
        plt.show()

interactive_output = widgets.interactive_output(
    show_image,
    {
        "upload": upload_button,
    }
)

display(widgets.HBox([upload_button, aux_output]), interactive_output)
